In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexing_distinct_tables = Elasticsearch()

In [2]:
tables = pd.read_csv('dataset/cleanDataTables2', delimiter=',', header=None, keep_default_na=False)
formattedTables = tables.iloc[:,:].values

In [ ]:
IDs = []
distinct_tables = []

for tables in tqdm(formattedTables):
        
    tablePgID = tables[0]
    tablePgTitle = str(tables[1])
    tableSectionTitle = str(tables[2])
    tableCaption = str(tables[3])
    tableHeader = str(tables[4])
    tableBody = str(tables[5])
    
    if tablePgID not in IDs:
       
        IDs.append(tablePgID)
        
        new_row = {"table_id": tablePgID,
               "table_page_title": tablePgTitle,
               "table_section_title": tableSectionTitle,
               "table_caption": tableCaption,
               "table_header": tableHeader,
               "table_body": tableBody}
    
        distinct_tables.append(new_row)

df_distinc_tables = pd.DataFrame(distinct_tables)  

In [ ]:
df_distinc_tables.to_csv('dataset_distinct_tables.csv',index=False)

In [2]:
distinc_tables = pd.read_csv('dataset/distinct_tables_allsignals.csv', delimiter=',', keep_default_na=False)

In [4]:
distinc_tables = distinc_tables.drop(distinc_tables[distinc_tables.table_page_title == ""].index)

In [7]:
def createIndexing():
    
    for i, row in distinc_tables.iterrows():
        
        tablePgID = row['table_id']
        tablePgTitle = row['table_page_title']
        tablePgFullText = row['table_page_full_text']
        tablePgMetaDescription = row['table_page_meta_description']
        tablePgSummary = row['table_page_summary']
        tablePgKeywords = row['table_page_keywords']
        tableSectionTitle = row['table_section_title']
        tableCaption = row['table_caption']
        tableHeader = row['table_header']
        tableBody = row['table_body']
        
        yield {
            "_index": "distinct_tables",
            "_type": "distinct_tables",
            "_source": {
                "tablePgID": tablePgID,
                "tablePgTitle": tablePgTitle,
                "tablePgFullText": tablePgFullText,
                "tablePgMetaDescription": tablePgMetaDescription,
                "tablePgSummary": tablePgSummary,
                "tablePgKeywords": tablePgKeywords,
                "tableSectionTitle": tableSectionTitle,
                "tableCaption": tableCaption,
                "tableHeader": tableHeader,
                "tableBody": tableBody
            }
            
        }

In [8]:
indexing_distinct_tables.indices.delete(index='distinct_tables')

{'acknowledged': True}

In [9]:
helpers.bulk(indexing_distinct_tables, createIndexing())

(85900, [])